# For generating demo data for genomes

In [ ]:
import logging
from pathlib import Path
import numpy as np
from roux.lib.io import read_ps,read_table,to_table,read_dict,to_dict

In [ ]:
## parameters
n=20
species_names=['yeast'] #['human']
force=True
# force=False

## Demo custom genome
### Download
      pyensembl install --release=100 --species=fly

In [ ]:
# from pyensembl.shell import collect_all_installed_ensembl_releases
# collect_all_installed_ensembl_releases()

from pyensembl import find_species_by_name
from pyensembl import EnsemblRelease
data = EnsemblRelease(
    release=100,
    species=find_species_by_name('drosophila_melanogaster'),
    )

### Subset to a chromosome

#### Annotations

In [ ]:
import pandas as pd

In [ ]:
df01=pd.read_csv(
    data.gtf_path,
    sep='\t',
    compression='gzip',
    header=2,
    names=["seqname",# - name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seqname must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly. See the example GFF output below.
"source",# - name of the program that generated this feature, or the data source (database or project name)
"feature",# - feature type name, e.g. Gene, Variation, Similarity
"start",# - Start position* of the feature, with sequence numbering starting at 1.
"end",# - End position* of the feature, with sequence numbering starting at 1.
"score",# - A floating point value.
"strand",# - defined as + (forward) or - (reverse).
"frame",# - One of '0', '1' or '2'. '0' indicates that the first base of the feature is the first base of a codon, '1' that the second base is the first base of a codon, and so on..
"attribute",# - A semicolon-separated list of tag-value pairs, providing additional information about each feature.
],
)
df01.head(1)

In [ ]:
df01['seqname'].unique()

In [ ]:
df1=df01.query("`seqname`=='4'") # shortest
len(df1)
df1.head(1)

In [ ]:
df1.to_csv('inputs/ann.gtf',index=False,header=False,sep='\t')

In [ ]:
df1['feature'].unique()

### Sequences

In [ ]:
from beditor.lib.utils import read_fasta
import gzip
from Bio import SeqIO
for molecule_type,p in {'RNA':data.transcript_fasta_paths[0],'Protein':data.protein_fasta_paths[0],}.items():
    with gzip.open(p, "rt") as handle:
        # s_=SeqIO.parse(handle,format='fasta')
        # for s in s_:
        #     break
        d1=read_fasta(handle,key_type='descr')

    print(len(d1))
    d1={k:v for k,v in d1.items() if 'chromosome:BDGP6.28:4:' in k}
    print(len(d1))

    to_fasta(d1,f'inputs/{molecule_type}.fa',molecule_type=molecule_type)

### Test by loading the subset

In [ ]:
from pyensembl import Genome
annots = Genome(
    reference_name='ref',
    annotation_name='ann',
    gtf_path_or_url='inputs/ann.gtf',
    transcript_fasta_paths_or_urls="inputs/RNA.fa",
    protein_fasta_paths_or_urls="inputs/Protein.fa",
)
# parse GTF and construct database of genomic features
annots.index()

## Input mutations

In [ ]:
parameters_list=[]
for mutation_format in ['protein','base']:
    for mutation_scan in ['regions','positions']: #[None]
        parameters_list.append(
            dict(
                mutation_format=mutation_format,
                mutation_scan=mutation_scan,
                output_path=f'inputs/mutations/{mutation_format}/{mutation_scan}.yml',
                n=n,
                force=force,
            )
        )
        # for species_name in species_names:
        #     parameters_list.append(
        #         dict(
        #             mutation_format=mutation_format,
        #             mutation_scan=mutation_scan,
        #             output_path=f'inputs_{species_name}/mutations/{mutation_format}/{mutation_scan}.yml',
        #             species_name=species_name,
        #             release=110,
        #             n=200,
        #             force=force,
        #         )
        #     )
len(parameters_list)

In [ ]:
from papermill import execute_notebook
for params in parameters_list:
    if not Path(params['output_path']).exists() or force:
        logging.info(params['output_path'])
        Path(params['output_path']).parent.mkdir(parents=True, exist_ok=True)
        outp=Path(params['output_path']).with_suffix('.ipynb')
        print(outp)
        execute_notebook(
            input_path='demo_data_for_run.ipynb',
            output_path=outp,
            parameters=params,
        )
    # break

### Mutations for base editing

In [ ]:
def map_mutations(
    df_,
    mutations,
    random=False,
    ):
    if random:
        np.random.seed(0)
        df_=df_.assign(
            mutation=np.random.choice(mutations,len(df_))
        )
    else:
        df_=df_.assign(
            mutation=lambda df : df[df_.columns.tolist()[0]].apply(lambda x: mutations),
        ).explode('mutation').log()
    print(df_['mutation'].value_counts().to_dict())
    return df_

In [ ]:
for ind in ["inputs/"]+[f"inputs_{s}/" for s in species_names]:
    for mutation_format in ['base','protein']:
        if mutation_format=='base':
            mutations=list('ATGC')
        else:
            from Bio.Data import IUPACData
            # Get a list of all amino acids
            mutations = list(IUPACData.protein_letters)
        df_=read_table(f"{ind}/mutations/{mutation_format}/positions.tsv")
        print(len(df_))
        df_=map_mutations(
            df_,
            mutations=mutations,
            random=False,
        )
        print(len(df_))
        to_table(df_,f'{ind}/mutations/{mutation_format}/points.tsv')

        cfg=read_dict(f"{ind}/mutations/{mutation_format}/positions.yml")
        cfg['run']['input_path']=cfg['run']['input_path'].replace('positions','points')
        cfg['run']['output_dir_path']=cfg['run']['output_dir_path'].replace('positions','points')
        to_dict(cfg,f"{ind}/mutations/{mutation_format}/points.yml")
        print(cfg)